In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from gensim.models import Doc2Vec
from collections import namedtuple
import gensim.utils
import re
import string
import requests
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim import similarities 

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [2]:
df = pd.read_csv(r'Hasil Crawl Halal MUI.csv')
df

,Unnamed: 0,id,no,nama,produsen,sertif,tanggal
0,0,109,21949,cyber park karawaci lt. 2 (head office),"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
1,1,121,21949,foodmart surabaya town square,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
2,2,95,21949,hypermart batam tanjung pinang,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
3,3,66,21949,hypermart bau bau,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
4,4,45,21949,hypermart ciputra world surabaya,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
...,...,...,...,...,...,...,...
401645,401645,3,7790,장군 olive oil laver,Asia food co.-korea,00100082470417,"08-october-2021"""
401646,401646,4,12758,한상궁 delicious scorched brown rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401647,401647,2,12758,한상궁 delicous scorched barley and rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401648,401648,1,12758,한상궁 delicous scorched rice,"Hando food co., ltd",00100086820118,"25-february-2022"""


In [3]:
df = df.drop_duplicates(subset=['nama'], keep='last')
df

,Unnamed: 0,id,no,nama,produsen,sertif,tanggal
0,0,109,21949,cyber park karawaci lt. 2 (head office),"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
1,1,121,21949,foodmart surabaya town square,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
2,2,95,21949,hypermart batam tanjung pinang,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
3,3,66,21949,hypermart bau bau,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
4,4,45,21949,hypermart ciputra world surabaya,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
...,...,...,...,...,...,...,...
401645,401645,3,7790,장군 olive oil laver,Asia food co.-korea,00100082470417,"08-october-2021"""
401646,401646,4,12758,한상궁 delicious scorched brown rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401647,401647,2,12758,한상궁 delicous scorched barley and rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401648,401648,1,12758,한상궁 delicous scorched rice,"Hando food co., ltd",00100086820118,"25-february-2022"""


In [4]:
@st.cache
def get_data():
    return pd.read_csv("Hasil Crawl Halal MUI.csv")

def cleandata(kalimat):
    kalimatbaru=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', kalimat)
    kalimatbaru=stopword.remove(kalimatbaru)
    return kalimatbaru

def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [5]:
lawdata=df
tanya = np.array(df.nama)
jawab=np.array(df.nama+'|'+df.produsen+'|'+df.sertif)
data=np.array(df.nama+' '+df.produsen)

2021-06-10 11:31:23.516 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [6]:
def get_response(q):
	highsim=0.0
	highnum=0
	for i,line in enumerate(data):
		if (type(line)==str):
			qlist=cleandata(q).lower().split()
			linelist=cleandata(line).lower().split()
			simnow=jaccard_similarity(qlist,linelist)
			if(simnow>highsim):
				highnum=i
				highsim=simnow
	return(highnum)  
	

In [7]:
def get_response2(q):
	highsim=0.0
	highnum=0
	for i,line in enumerate(data):
		if (type(line)==str):
			qlist=cleandata(q).lower().split()
			linelist=cleandata(line).lower().split()
			simnow=jaccard_similarity(qlist,linelist)
			if(simnow>highsim):
				highnum=i
				highsim=simnow
	return(highsim)  

In [8]:
docs2 = pd.read_csv('Merge Indomaret.csv')
docs2

,Unnamed: 0,index,nama_produk,komposisi
0,0,1,Pristine 8+ Air Mineral 600Ml,NaN
1,1,2,Pristine 8+ Air Mineral 400Ml,NaN
2,2,3,Crystalline Air Mineral 600Ml,NaN
3,3,4,Le Minerale Air Mineral 1500Ml,"Calcium, magnesium, sodium, potassium, nitrate..."
4,4,5,Le Minerale Air Mineral 600Ml,"Calcium, magnesium, sodium, potassium, nitrate..."
...,...,...,...,...
1328,1328,1329,Blue Band Margarine Serbaguna 250G,"Minyak nabati, air, garam, pengemulsi monodigl..."
1329,1329,1330,Dougo Canola Oil 1000Ml,biji bunga canola.BPOM RI ML 210216001071Diimp...
1330,1330,1331,Blue Band Margarine Serbaguna 200G,"Minyak nabati, air, garam, pengemulsi monodigl..."
1331,1331,1332,Blue Band Margarine Cake & Cookie 200G,NaN


In [9]:
docs2 = docs2.drop(columns=['Unnamed: 0'])
docs2

,index,nama_produk,komposisi
0,1,Pristine 8+ Air Mineral 600Ml,NaN
1,2,Pristine 8+ Air Mineral 400Ml,NaN
2,3,Crystalline Air Mineral 600Ml,NaN
3,4,Le Minerale Air Mineral 1500Ml,"Calcium, magnesium, sodium, potassium, nitrate..."
4,5,Le Minerale Air Mineral 600Ml,"Calcium, magnesium, sodium, potassium, nitrate..."
...,...,...,...
1328,1329,Blue Band Margarine Serbaguna 250G,"Minyak nabati, air, garam, pengemulsi monodigl..."
1329,1330,Dougo Canola Oil 1000Ml,biji bunga canola.BPOM RI ML 210216001071Diimp...
1330,1331,Blue Band Margarine Serbaguna 200G,"Minyak nabati, air, garam, pengemulsi monodigl..."
1331,1332,Blue Band Margarine Cake & Cookie 200G,NaN


In [10]:
docs2 = docs2.drop_duplicates(keep='last')
docs2

,index,nama_produk,komposisi
0,1,Pristine 8+ Air Mineral 600Ml,NaN
1,2,Pristine 8+ Air Mineral 400Ml,NaN
2,3,Crystalline Air Mineral 600Ml,NaN
3,4,Le Minerale Air Mineral 1500Ml,"Calcium, magnesium, sodium, potassium, nitrate..."
4,5,Le Minerale Air Mineral 600Ml,"Calcium, magnesium, sodium, potassium, nitrate..."
...,...,...,...
1328,1329,Blue Band Margarine Serbaguna 250G,"Minyak nabati, air, garam, pengemulsi monodigl..."
1329,1330,Dougo Canola Oil 1000Ml,biji bunga canola.BPOM RI ML 210216001071Diimp...
1330,1331,Blue Band Margarine Serbaguna 200G,"Minyak nabati, air, garam, pengemulsi monodigl..."
1331,1332,Blue Band Margarine Cake & Cookie 200G,NaN


In [11]:
jawab[get_response("Nutella Jam Spread Hazelnut With Cocoa")]

'Nutella spread|Pt. trans retail indonesia|00160095720519'

In [12]:
items = []

In [14]:
for i, row in docs2.iterrows():
    q1 = row['nama_produk']
    items.append(q1+"|"+jawab[get_response(q1)]+"|"+str(get_response2(q1)))

In [15]:
print(items)

['Pristine 8+ Air Mineral 600Ml|Ian air mineral 600ml|Cv. litara jaya|08120011160920|0.3333333333333333', 'Pristine 8+ Air Mineral 400Ml|Air mineral botol aqua|Kampung kecil|00160097550719|0.25', 'Crystalline Air Mineral 600Ml|Ian air mineral 600ml|Cv. litara jaya|08120011160920|0.375', 'Le Minerale Air Mineral 1500Ml|Air mineral le minerale|Cv. oto boga jaya|00160117390321|0.4444444444444444', 'Le Minerale Air Mineral 600Ml|Air mineral le minerale|Cv. oto boga jaya|00160117390321|0.4444444444444444', 'Club Air Mineral 600Ml|Club air mineral|Pt. trans retail indonesia|00160095720519|0.375', 'Nestle Air Mineral Pure Life 600Ml|Air mineral nestle pure life|Pt. kuliner sukses inovatif|00160101680120|0.5', 'Club Air Mineral 1500Ml|Club air mineral|Pt. trans retail indonesia|00160095720519|0.375', 'Aqua Galon Kosong + Isi Ulang 19L|Air isi ulang panji galon|Panji galon|02160016671120|0.3333333333333333', 'Aqua Air Mineral Click N Go Kids 450Ml|Air mineral botol aqua|Kampung kecil|0016009755

In [16]:
from pandas import DataFrame
new_df = DataFrame(items,columns=['Produk'])
new_df

,Produk
0,Pristine 8+ Air Mineral 600Ml|Ian air mineral ...
1,Pristine 8+ Air Mineral 400Ml|Air mineral boto...
2,Crystalline Air Mineral 600Ml|Ian air mineral ...
3,Le Minerale Air Mineral 1500Ml|Air mineral le ...
4,Le Minerale Air Mineral 600Ml|Air mineral le m...
...,...
1328,Blue Band Margarine Serbaguna 250G|Blue band m...
1329,Dougo Canola Oil 1000Ml|Rbd canola oil|Pt. kim...
1330,Blue Band Margarine Serbaguna 200G|Blue band m...
1331,Blue Band Margarine Cake & Cookie 200G |Blue b...


In [17]:
new_df.to_csv(r'Match dengan Jaccard Baru.csv', index = False)

# Processing hasil Jaccard

In [18]:
match = pd.read_csv(r'Match dengan Jaccard Baru.csv')
match = match.drop_duplicates()
match

,Produk
0,Pristine 8+ Air Mineral 600Ml|Ian air mineral ...
1,Pristine 8+ Air Mineral 400Ml|Air mineral boto...
2,Crystalline Air Mineral 600Ml|Ian air mineral ...
3,Le Minerale Air Mineral 1500Ml|Air mineral le ...
4,Le Minerale Air Mineral 600Ml|Air mineral le m...
...,...
1328,Blue Band Margarine Serbaguna 250G|Blue band m...
1329,Dougo Canola Oil 1000Ml|Rbd canola oil|Pt. kim...
1330,Blue Band Margarine Serbaguna 200G|Blue band m...
1331,Blue Band Margarine Cake & Cookie 200G |Blue b...


In [19]:
new = match.Produk.str.split("|", n = 5, expand=True,)

In [20]:
match["produkindo"]= new[0]
match["produkmui"] = new[1]
match["produsen"] = new[2]
match["sertif"] = new[3]
match["nilai"] = new[4]
match

,Produk,produkindo,produkmui,produsen,sertif,nilai
0,Pristine 8+ Air Mineral 600Ml|Ian air mineral ...,Pristine 8+ Air Mineral 600Ml,Ian air mineral 600ml,Cv. litara jaya,08120011160920,0.3333333333333333
1,Pristine 8+ Air Mineral 400Ml|Air mineral boto...,Pristine 8+ Air Mineral 400Ml,Air mineral botol aqua,Kampung kecil,00160097550719,0.25
2,Crystalline Air Mineral 600Ml|Ian air mineral ...,Crystalline Air Mineral 600Ml,Ian air mineral 600ml,Cv. litara jaya,08120011160920,0.375
3,Le Minerale Air Mineral 1500Ml|Air mineral le ...,Le Minerale Air Mineral 1500Ml,Air mineral le minerale,Cv. oto boga jaya,00160117390321,0.4444444444444444
4,Le Minerale Air Mineral 600Ml|Air mineral le m...,Le Minerale Air Mineral 600Ml,Air mineral le minerale,Cv. oto boga jaya,00160117390321,0.4444444444444444
...,...,...,...,...,...,...
1328,Blue Band Margarine Serbaguna 250G|Blue band m...,Blue Band Margarine Serbaguna 250G,Blue band margarine,Pt. trans retail indonesia,00160095720519,0.3333333333333333
1329,Dougo Canola Oil 1000Ml|Rbd canola oil|Pt. kim...,Dougo Canola Oil 1000Ml,Rbd canola oil,Pt. kimia farma,00080102960320,0.25
1330,Blue Band Margarine Serbaguna 200G|Blue band m...,Blue Band Margarine Serbaguna 200G,Blue band margarine,Pt. trans retail indonesia,00160095720519,0.3333333333333333
1331,Blue Band Margarine Cake & Cookie 200G |Blue b...,Blue Band Margarine Cake & Cookie 200G,Blue band margarine,Pt. trans retail indonesia,00160095720519,0.2727272727272727


In [21]:
match = match.sort_values(['nilai'], ascending=False)
match

,Produk,produkindo,produkmui,produsen,sertif,nilai
510,Dua Kelinci Kacang Polong Polongmas Ayam Bawan...,Dua Kelinci Kacang Polong Polongmas Ayam Bawan...,Dua kelinci polongmas kacang polong rasa ayam ...,Pt. dua kelinci,00100017571001,0.7777777777777778
419,Dua Kelinci Jagung Presto Pedas Manis 70G|Dua ...,Dua Kelinci Jagung Presto Pedas Manis 70G,Dua kelinci jagung presto rasa pedas manis,Pt. dua kelinci,00100017571001,0.75
418,Dua Kelinci Jagung Presto Bawang Putih 70G |Du...,Dua Kelinci Jagung Presto Bawang Putih 70G,Dua kelinci jagung presto rasa bawang putih,Pt. dua kelinci,00100017571001,0.75
514,Dua Kelinci Kacang Sangrai 180G|Dua kelinci ka...,Dua Kelinci Kacang Sangrai 180G,Dua kelinci kacang sangrai,Pt. dua kelinci,00100017571001,0.6666666666666666
575,Frisian Flag Susu Bubuk Full Cream 400G|Frisia...,Frisian Flag Susu Bubuk Full Cream 400G,Frisian flag susu bubuk full cream,Pt. frisian flag indonesia,00040040520606,0.6666666666666666
...,...,...,...,...,...,...
377,Loacker Wafer Quadratini Napolitaner 250G|Wafe...,Loacker Wafer Quadratini Napolitaner 250G,Wafer cone,Ud. aldy,00290032980105,0.125
378,Loacker Wafer Quadratini Napolitaner 125G|Wafe...,Loacker Wafer Quadratini Napolitaner 125G,Wafer cone,Ud. aldy,00290032980105,0.125
1108,Kolln Muesli Strawbry Yogurt 375g|Le qurma yog...,Kolln Muesli Strawbry Yogurt 375g,Le qurma yogurt kurma,Le qurma,03120026520719\t\t\t,0.125
15,Aqua Galon 19L (Isi Ulang)|Pegangan galon|Asia...,Aqua Galon 19L (Isi Ulang),Pegangan galon,"Asia, cv.",07170018941013,0.125


In [22]:
match.to_csv(r'Match Lengkap Coba 2.csv')